In [1]:
import sys
path = '/Users/xiehao/Desktop/workspace/X2AI/sugar_ai/'
if path not in sys.path:
    sys.path.append(path)

import pandas as pd
from DataBuilder.hisugar.crawler import HigSugarCrawler
from DataBuilder.future_bar1d.builder import FutureBar1dBuilder
from Base import DBFile, DBSQL
from AIBots.SentimentalBot.robot import SentimentalBot

# 设置参数
start_date = "2025-06-11"
end_date = "2025-06-14"

# HigSugarCrawler(
#     start_date=start_date,
#     end_date=end_date,
#     db=DBFile(),
# ).crawl()

# FutureBar1dBuilder(
#     start_date=start_date,
#     end_date=end_date,
#     db=DBFile(),
# ).build()

# AI 研究
bot = SentimentalBot(db=DBFile(), start_date=start_date, end_date=end_date)
bot.analyzing()

/Users/xiehao/Desktop/workspace/X2AI/sugar_ai/AIBots/SentimentalBot/robot.py:258: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["rating"] = df["rating"].replace({"震荡": 0, "上涨": 1, "下跌": -1})
